In [ ]:
import os


import sys
sys.path.insert(0,os.getcwd())

print(os.getcwd())
print(sys.path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from typing import List, Dict

from pytrends.request import TrendReq

from tqdm import tqdm
from itertools import islice
from IPython.display import display
import time

from warnings import warn

In [ ]:
import json

# Load the JSON file
with open('remedies.json', 'r') as f:
    remedies = json.load(f)

In [ ]:
remedies = [r for r in remedies if r.get('VERNAC') is not None]
len(remedies)

In [ ]:
remedies = [r for i, r in enumerate(remedies) if r.get('CNAME') is not None]
len(remedies)


In [ ]:
remedies = list({r['TAXON']: r for r in remedies}.values())

In [ ]:
len(remedies)

In [ ]:
remedies_names = [remedy['TAXON'][0] for remedy in remedies]

In [23]:
remedies_names = []
for remedy in remedies:
    if remedy.get('TAXON'):
        if isinstance(remedy['TAXON'], list):
            remedies_names.append(remedy['TAXON'][0])
        else:
            remedies_names.append(remedy['TAXON'])

print(remedies_names)


['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox', 'Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Amomum sp', 'Andrographis paniculata', 'Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica', 'Bupleurum falcatum', 'Calendula officinalis', 'Cassia senna', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa', 'Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Coptis sp', 'Crataegus monogyna', 'Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Echinacea purpurea', 'Ephedra sinica', 'Ephedra sp', 'Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza ralensis', 'Glycyrrhiza uralensis', 'Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melaleuca alternifolia', 'Melissa officinalis', 'Mentha piperita', 'Ocimum sanctum', 'Oenothera bien

In [24]:
remedies = ['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox', 'Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Andrographis paniculata', 'Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica', 'Bupleurum falcatum', 'Calendula officinalis', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa', 'Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Crataegus monogyna', 'Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Ephedra sinica', 'Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza uralensis', 'Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melissa officinalis', 'Mentha piperita', 'Ocimum sanctum', 'Oenothera biennis', 'Paeonia lactiflora', 'Panax ginseng', 'Piper methysticum', 'Plantago asiatica', 'Plantago ovata', 'Platycodon grandiflorum', 'Polygala senega', 'Rauvolfia serpentina', 'Rhamnus frangula', 'Rhamnus purshiana', 'Rheum officinale', 'Rheum palmatum', 'Rosa laevigata', 'Rosa sp', 'Sambucus nigra', 'Serenoa repens', 'Silybum marianum', 'Syzygium aromaticum', 'Thymus vulgaris', 'Thymus zygis', 'Urtica dioica', 'Urtica urens', 'Valeriana officinalis', 'Zingiber officinale']

groups = [remedies[i:i+5] for i in range(0, len(remedies), 5)]

print(groups)


[['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox'], ['Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Andrographis paniculata'], ['Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica'], ['Bupleurum falcatum', 'Calendula officinalis', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa'], ['Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Crataegus monogyna'], ['Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Ephedra sinica'], ['Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza uralensis'], ['Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melissa officinalis', 'Mentha piperita'], ['Ocimum sanctum', 'Oenothera biennis', 'Paeonia lactiflora', 'Panax ginseng', 'Piper methysticum'], ['Plantago asiatica', 'Plantago ovata', 'Pla

In [25]:
def break_into_groups(lst, n):
    """
    This function breaks a list into n-sized groups.
    """
    return [lst[i:i + n] for i in range(0, len(lst), n)]

def rank_by_popularity(groups, debug=False):
    """
    This function ranks a list of groups by the popularity of the herbs in them.
    """
    url = 'https://herb-ranking.herokuapp.com/rank'
    headers = {'Content-Type': 'application/json'}
    rankings = []

    for group in groups:
        data = {'herbs': group}
        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 200:
            ranking = response.json()['ranking']
            rankings.append(ranking)

            if debug:
                print(group)
                print(ranking)
                print('---')
        else:
            print(f"Error {response.status_code} when ranking {group}")

    return rankings

def pytrend_score_group(groups: List[List[str]], print_table: bool = False) -> Dict[str, int]:
    pt = TrendReq(retries=5)
    scores = {remedy: 0 for remedy in set([remedy for group in groups for remedy in group])}
    
    for group in groups:
        pt.build_payload(group)
        tmp_df = pt.interest_over_time()
        tmp_df = tmp_df[group].mean()
        ranks = tmp_df.rank().to_dict()
        for remedy, rank in ranks.items():
            scores[remedy] += rank
            
if print_table:
        df = pd.DataFrame.from_dict(scores, orient='index', columns=['score'])
        df.index.name = 'remedy'
        df.reset_index(inplace=True)
        df = df.sort_values(by=['score','remedy'], ascending=False)
        display(df)
        
return scores           

NameError: name 'print_table' is not defined

In [26]:
herbs = ['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox', 'Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Andrographis paniculata', 'Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica', 'Bupleurum falcatum', 'Calendula officinalis', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa', 'Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Crataegus monogyna', 'Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Ephedra sinica', 'Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza uralensis', 'Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melissa officinalis', 'Mentha piperita', 'Ocimum sanctum', 'Oenothera biennis', 'Paeonia lactiflora', 'Panax ginseng', 'Piper methysticum', 'Plantago asiatica', 'Plantago ovata', 'Platycodon grandiflorum', 'Polygala senega', 'Rauvolfia serpentina', 'Rhamnus frangula', 'Rhamnus purshiana', 'Rheum officinale', 'Rheum palmatum', 'Rosa laevigata', 'Rosa sp', 'Sambucus nigra', 'Serenoa repens', 'Silybum marianum', 'Syzygium aromaticum', 'Thymus vulgaris', 'Thymus zygis', 'Urtica dioica', 'Urtica urens', 'Valeriana officinalis', 'Zingiber officinale']

herbs = list(set(herbs))

GROUP_SIZE = 5

for herb, score in ranked_group.items():
    herb_score[herb] += ranked_group[herb]
    herb_i[herb] += 1
            
# create a dataframe from herb_score
df = pd.DataFrame.from_dict(herb_score, orient='index', columns=['score'])
df.index.name = 'herb'
df.reset_index(inplace=True)

# sort the dataframe by score and herb
df = df.sort_values(by=['score','herb'], ascending=False)

# get the sorted herbs
sorted_herbs = df['herb'].tolist()

# break the sorted herbs into groups
groups = break_into_groups(sorted_herbs,GROUP_SIZE)
    
# create a new dictionary to store the scores for each group
group_scores = {f'Group {i}': 0 for i in range(1, len(groups)+1)}

# assign the scores for each group based on the scores of the herbs in the group
for i, group in enumerate(groups):
    group_score = sum([herb_score[herb] for herb in group])
    group_scores[f'Group {i+1}'] = group_score

# create a dataframe from group_scores
df_groups = pd.DataFrame.from_dict(group_scores, orient='index', columns=['score'])
df_groups.index.name = 'group'
df_groups.reset_index(inplace=True)

# sort the dataframe by score and group
df_groups = df_groups.sort_values(by=['score','group'], ascending=False)

# display the final result
print(df_groups)


KeyError: 'herb1'

In [27]:
df_groups

NameError: name 'df_groups' is not defined

In [2]:
from pytrends.request import TrendReq
pytrend = TrendReq()

GROUP_SIZE = 5
groups = [['Aesculus hippocastanum', 'Allium cepa', 'Allium sativum', 'Aloe africana', 'Aloe ferox'],
          ['Aloe sp', 'Aloe spicata', 'Aloe vera', 'Althaea officinalis', 'Andrographis paniculata'],
          ['Angelica sinensis', 'Arctostaphylos uva-ursi', 'Astragalus membranaceus', 'Astragalus mongholicus', 'Brucea javanica'],
          ['Bupleurum falcatum', 'Calendula officinalis', 'Centella asiatica', 'Chrysanthemum parthenium', 'Cimicifuga racemosa'],
          ['Cinnamomum cassia', 'Cinnamomum verum', 'Coptis chinensis', 'Coptis japonica', 'Crataegus monogyna'],
          ['Crataegus sp', 'Curcuma longa', 'Echinacea angustifolia', 'Echinacea pallida', 'Ephedra sinica'],
          ['Eucalyptus globulus', 'Eucalyptus sp', 'Ginkgo biloba', 'Glycyrrhiza glabra', 'Glycyrrhiza uralensis'],
          ['Hamamelis virginiana', 'Hypericum perforatum', 'Matricaria chamomilla', 'Melissa officinalis', 'Mentha piperita'],
          ['Ocimum sanctum', 'Oenothera biennis', 'Paeonia lactiflora', 'Panax ginseng', 'Piper methysticum'],
          ['Plantago asiatica', 'Plantago ovata', 'Platycodon grandiflorum', 'Polygala senega', 'Rauvolfia serpentina'],
          ['Rhamnus frangula', 'Rhamnus purshiana', 'Rheum officinale', 'Rheum palmatum', 'Rosa laevigata'],
          ['Rosa sp', 'Sambucus nigra', 'Serenoa repens', 'Silybum marianum', 'Syzygium aromaticum'],
          ['Thymus vulgaris', 'Thymus zygis', 'Urtica dioica', 'Urtica urens', 'Valeriana officinalis'],
          ['Zingiber officinale']]
        
herb_i = {herb: 0 for herb in herbs}

for herb, score in ranked_group.items():
    herb_score[herb] += ranked_group[herb]
    herb_i[herb] += 1

ROUNDS = 10

for rnd in tqdm(range(ROUNDS)):
    
    # we need to iterate the groups
    for group in groups:

        #rank the herbs inside the group
        try:
            ranked_group = rank_by_popularity([group], debug=False)
        except:
            
            try:
                warn("Max retries archived, let's wait 30 seconds to keep the process")
                time.sleep(60.1)
                ranked_group = rank_by_popularity([group], debug=False)
            
            except:
                break
                
            time.sleep(0.5)
        
        # initialize herb scores to 0
herb_score = {herb: 0 for herb in herbs}

# rank herbs
herb_i = {herb: 0 for herb in herbs}
for i, herb in enumerate(ranked_group):
    score = GROUP_SIZE - i
    herb_score.setdefault(herb, 0)
    herb_score[herb] += score
    herb_i[herb] += 1

# create DataFrame and sort by score and herb
df = pd.DataFrame.from_dict(herb_score, orient='index', columns=['score'])
df.index.name = 'herb'
df.reset_index(inplace=True)
df = df.sort_values(by=['score', 'herb'], ascending=False)

# store sorted list of herbs in sorted_herbs
sorted_herbs = list(df['herb'])


   
    
   
        


NameError: name 'herbs' is not defined

In [17]:

# create a DataFrame from the herb_score dictionary
df = pd.DataFrame(list(herb_score.items()), columns=['herb', 'score'])

# sort the DataFrame by score and herb
df = df.sort_values(by=['score', 'herb'], ascending=[False, True])

# store the sorted list of herbs in sorted_herbs
sorted_herbs = df['herb'].tolist()


In [18]:
sorted_herbs


['herb1',
 'Aesculus hippocastanum',
 'Allium cepa',
 'Allium sativum',
 'Aloe africana',
 'Aloe ferox',
 'Aloe sp',
 'Aloe spicata',
 'Aloe vera',
 'Althaea officinalis',
 'Andrographis paniculata',
 'Angelica sinensis',
 'Arctostaphylos uva-ursi',
 'Astragalus membranaceus',
 'Astragalus mongholicus',
 'Brucea javanica',
 'Bupleurum falcatum',
 'Calendula officinalis',
 'Centella asiatica',
 'Chrysanthemum parthenium',
 'Cimicifuga racemosa',
 'Cinnamomum cassia',
 'Cinnamomum verum',
 'Coptis chinensis',
 'Coptis japonica',
 'Crataegus monogyna',
 'Crataegus sp',
 'Curcuma longa',
 'Echinacea angustifolia',
 'Echinacea pallida',
 'Ephedra sinica',
 'Eucalyptus globulus',
 'Eucalyptus sp',
 'Ginkgo biloba',
 'Glycyrrhiza glabra',
 'Glycyrrhiza uralensis',
 'Hamamelis virginiana',
 'Hypericum perforatum',
 'Matricaria chamomilla',
 'Melissa officinalis',
 'Mentha piperita',
 'Ocimum sanctum',
 'Oenothera biennis',
 'Paeonia lactiflora',
 'Panax ginseng',
 'Piper methysticum',
 'Planta